In [1]:
import torch
import pandas as pd
from collections import defaultdict
import random

In [2]:
from utils.data.dataloaders import make_mnist_dataloaders
from utils.models.supernet import SuperNet 
from utils.trainer import Trainer

In [3]:
data_dir = './data/'
models_dir = './models/'
results_dir = './results/'
n_epoch = 30
batch_size = 512
num_workers = 0
lr = 1e-2
weight_decay = 0.0
strategies = ['random', 'mean', 'sum', 'dropout_0.1', 'dropout_0.3', 'dropout_0.5', 'dropout_0.7']
seed = 0

In [4]:
random.seed(seed)

In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [6]:
train_dataloader, val_dataloader, test_dataloader = make_mnist_dataloaders(data_dir=data_dir, random_state=seed, batch_size=batch_size, num_workers=num_workers)

In [7]:
optimizers = { 'sgd_m': lambda parameters, weight_decay, lr: torch.optim.SGD(parameters, lr=lr, weight_decay=weight_decay, momentum=0.9),
               'sgd': torch.optim.SGD,
               'adam': torch.optim.Adam}

default_val_params = {'test_dataloader': val_dataloader, 'show_results': False, 'verbose': False, 'log': True}
default_test_params = {'test_dataloader': test_dataloader, 'show_results': False, 'verbose': False, 'log': True}

In [8]:
accuracies_valid = defaultdict(list)
accuracies_test = defaultdict(list)
for optimizer in optimizers:
    for strategy in strategies:    
        default_train_params = {'train_dataloader': train_dataloader, 'val_dataloader': val_dataloader, 'n_epoch': n_epoch, 
                        'optim': optimizers[optimizer], 'weight_decay': weight_decay, 'schedul': None, 'loss': torch.nn.CrossEntropyLoss, 
                        'weighted': True, 'lr': lr, 'show_results': False, 'saved_models_dir': models_dir, 'verbose': False, 
                        'early_stopping': True, 'max_gap': 2, 'gamma': None}
        
        #обучение суперсети для каждой стратегии обучения
        model = SuperNet(strategy=strategy)
        trainer = Trainer(model, device=device, label_names=list(range(10)), model_name=f'supernet_{strategy}_{optimizer}')
        trainer.train(**default_train_params)
    
        #сэмплирование
        model1 = model.sampler(1)
        model2 = model.sampler(2)
    
        #вычисление метрик сэмплов суперсети
        trainer1 = Trainer(model1, device=device, label_names=list(range(10)), model_name=f'sample1_{strategy}_{optimizer}')
        trainer2 = Trainer(model2, device=device, label_names=list(range(10)), model_name=f'sample2_{strategy}_{optimizer}')
    
        _, _, history = trainer1.test(**default_val_params)
        accuracies_valid[f'{strategy} {optimizer}'].append(history['accuracy'])
        _, _, history = trainer1.test(**default_test_params)
        accuracies_test[f'{strategy} {optimizer}'].append(history['accuracy'])
  
        _, _, history = trainer2.test(**default_val_params)
        accuracies_valid[f'{strategy} {optimizer}'].append(history['accuracy'])
        _, _, history = trainer2.test(**default_test_params)
        accuracies_test[f'{strategy} {optimizer}'].append(history['accuracy'])
    
        #случайная инициализация сэмплов суперсети
        model1.random_init()
        model2.random_init()  
    
        #обучение сэмплов суперсети с нуля
        trainer1.train(**default_train_params)
        trainer2.train(**default_train_params)
    
        #вычисление метрик сэмплов суперсети
        _, _, history = trainer1.test(**default_val_params)
        accuracies_valid[f'{strategy} {optimizer}'].append(history['accuracy'])
        _, _, history = trainer1.test(**default_test_params)
        accuracies_test[f'{strategy} {optimizer}'].append(history['accuracy'])
    
        _, _, history = trainer2.test(**default_val_params)
        accuracies_valid[f'{strategy} {optimizer}'].append(history['accuracy'])
        _, _, history = trainer2.test(**default_test_params)
        accuracies_test[f'{strategy} {optimizer}'].append(history['accuracy'])

In [9]:
model_names = ['sample_1 before', 'sample_2 before', 'sample_1 after', 'sample_2 after']
results_valid = pd.DataFrame(model_names, columns=['model'])
results_test = pd.DataFrame(model_names, columns=['model'])
for optimizer in optimizers:
    for strategy in strategies:
        results_valid[f'{strategy} {optimizer}'] = accuracies_valid[f'{strategy} {optimizer}']
        results_test[f'{strategy} {optimizer}'] = accuracies_test[f'{strategy} {optimizer}']

In [10]:
results_valid.to_excel(f'{results_dir}/тестовая_выборка.xls', index=False)

In [11]:
results_test.to_excel(f'{results_dir}/отложенная_выборка.xls', index=False)